In [1]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jl
import datetime as dt
import requests

from icecream import ic

import os
from dotenv import load_dotenv

from ta import add_all_ta_features
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochRSIIndicator

1. go to that github code and just switch it for all my etfs
2. change to paper trading api

In [2]:
load_dotenv()
TOKEN = os.getenv('TOKEN')

## Tradier Daily w/ Technical Analysis

In [ ]:
tickers = ['SPY', 'QQQ']
d = {}

for i in range(len(tickers)): 

    response = requests.get('https://api.tradier.com/v1/markets/history',
        params={'symbol': tickers[i], 'interval': 'daily', 'start': '2019-01-01', 'end': '2023-01-01'},
        headers={'Authorization': TOKEN, 'Accept': 'application/json'}
    )
    json_response = response.json()
    #print("Response Status Code:", response.status_code)
    i = i + 1
    print("progress", i)
    for symbol in tickers:
        d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

        indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
        d[symbol]['macd'] = indicator_macd.macd()
        d[symbol]['macd_diff'] = indicator_macd.macd_diff()
        d[symbol]['macd_signal'] = indicator_macd.macd_signal()

        indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
        d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
        d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
        d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
        d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
        d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

        indicator_vwap = VolumeWeightedAveragePrice(
            high = d[symbol]['high'],
            low = d[symbol]['low'],
            close = d[symbol]['close'],
            volume = d[symbol]['volume'],
            window=14, fillna=True)
        d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

        indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
        d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
        d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
        d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

In [3]:
# # create watchlist
# response = requests.post('https://api.tradier.com/v1/watchlists',
#     data={'name': 'CNBC_INDEX_LIST', 'symbols': 'SPY,QQQ,IWM,DIA,VTI,MDY,DBC,FEZ,OEF,IWF,IWD,PFF,VOO,IJH,IWO,IWN,ACWI,IEMG'},
#     headers={'Authorization': TOKEN, 'Accept': 'application/json'}
# )
# json_response = response.json()

In [4]:
# pull watchlist

response = requests.get('https://api.tradier.com/v1/watchlists/CNBC_INDEX_LIST',
    params={},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print('Status Code:', response.status_code)

Status Code: 200


In [5]:
#tickers_df = pd.DataFrame.from_records(json_response['watchlist']['items']['item'])
#tickers = list(pd.read_csv('data/etf_list.csv').Symbol)

In [6]:
response = requests.get('https://api.tradier.com/beta/markets/fundamentals/financials',
    params={'symbols': 'AAPL'},
    headers={'Authorization': TOKEN, 'Accept': 'application/json'}
)
json_response = response.json()
print(response.status_code)
#print(json_response)

200


## EOD 

In [214]:
def get_etf_tickers(): 
    
    # pulls all tickers of ETFs on NYSE or NASDAQ

    r = requests.get('https://eodhistoricaldata.com/api/exchange-symbol-list/US', 
        params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
        )
    data = r.json()
    r.close()

    df = pd.DataFrame(data)
    df = df[
        (df.Type == 'ETF') &
        ((df.Exchange == 'NYSE ARCA') |
        (df.Exchange == 'NASDAQ'))
        ]

    df.index = df.Code
    df.drop('Code', axis = 1, inplace=True)
    ticker_list = list(df.index)
    return ticker_list

In [223]:
tickers = get_etf_tickers()

In [224]:
df = pd.DataFrame(tickers)

df.to_csv('data/tickers.csv')

In [216]:
def get_historical_price(tickers, data_type):

    # pulls historical eod or intraday OLHC and volume

    d = {}

    for i in range(len(tickers)): 

        r = requests.get('https://eodhistoricaldata.com/api' + '/' + data_type + '/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        r.close()

        # ADD PROGRESS BAR
        
        for symbol in tickers:
            d[symbol] = pd.DataFrame.from_records(data)

            indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
            d[symbol]['macd'] = indicator_macd.macd()
            d[symbol]['macd_diff'] = indicator_macd.macd_diff()
            d[symbol]['macd_signal'] = indicator_macd.macd_signal()

            indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
            d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
            d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
            d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
            d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
            d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

            indicator_vwap = VolumeWeightedAveragePrice(
                high = d[symbol]['high'],
                low = d[symbol]['low'],
                close = d[symbol]['close'],
                volume = d[symbol]['volume'],
                window=14, fillna=True)
            d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

            indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
            d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
            d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
            d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

    return d

In [217]:
tickers = ['SPY', 'QQQ']

intraday_data = get_historical_price(tickers, 'intraday')

daily_data = get_historical_price(tickers, 'eod')

In [218]:
intraday_df = pd.concat(data_intraday.values(), axis=1, keys=data_intraday.keys())
intraday_dask_df = dd.from_pandas(df, npartitions=6)

daily_df = pd.concat(data_daily.values(), axis=1, keys=data_daily.keys())
daily_dask_df = dd.from_pandas(df, npartitions=6)

In [246]:
def get_fundementals(tickers): 

    d = {}

    for i in range(len(tickers)): 

        r = requests.get('https://eodhistoricaldata.com/api/fundamentals/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        
        r.close()

        for symbol in tickers:

            d[symbol] = pd.DataFrame.from_records(data)

            general = pd.DataFrame(d[symbol]['General'])
            holdings = pd.DataFrame(d[symbol]['ETF_Data']['Holdings']).T
            performance = pd.DataFrame(list(d[symbol]['ETF_Data']['Performance'].items()))
            performance.index = d[symbol]['ETF_Data']['Performance'].keys()
            performance = performance.drop(0, axis = 1)

In [247]:
tickers = ['SPY', 'QQQ']
fundemental_data = get_fundementals(tickers)

In [248]:
def test(tickers): 

    d = {}

    for i in range(len(tickers)): 

        r = requests.get('https://eodhistoricaldata.com/api/fundamentals/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        
        r.close()
        return data

In [249]:
tickers = ['SPY', 'QQQ']
fundemental_data = test(tickers)

In [257]:
pd.DataFrame(fundemental_data['ETF_Data']).columns

Index(['ISIN', 'Company_Name', 'Company_URL', 'ETF_URL', 'Domicile',
       'Index_Name', 'Yield', 'Dividend_Paying_Frequency', 'Inception_Date',
       'Max_Annual_Mgmt_Charge', 'Ongoing_Charge', 'Date_Ongoing_Charge',
       'NetExpenseRatio', 'AnnualHoldingsTurnover', 'TotalAssets',
       'Average_Mkt_Cap_Mil', 'Market_Capitalisation', 'Asset_Allocation',
       'World_Regions', 'Sector_Weights', 'Fixed_Income', 'Holdings_Count',
       'Top_10_Holdings', 'Holdings', 'Valuations_Growth', 'MorningStar',
       'Performance'],
      dtype='object')

In [271]:
pd.DataFrame(fundemental_data['ETF_Data']['Asset_Allocation'])
pd.DataFrame(fundemental_data['ETF_Data']['World_Regions'])
pd.DataFrame(fundemental_data['ETF_Data']['Sector_Weights'])
pd.DataFrame(fundemental_data['ETF_Data']['Sector_Weights'])
pd.DataFrame(fundemental_data['ETF_Data']['Fixed_Income'])
pd.Series(fundemental_data['ETF_Data']['Holdings_Count'])
pd.DataFrame(fundemental_data['ETF_Data']['Top_10_Holdings'].values(), index = fundemental_data['ETF_Data']['Top_10_Holdings'].keys())
pd.DataFrame(fundemental_data['ETF_Data']['Holdings'].values(), index = fundemental_data['ETF_Data']['Holdings'].keys())

,Code,Exchange,Name,Sector,Industry,Country,Region,Assets_%
AAPL.US,AAPL,US,Apple Inc,Technology,Consumer Electronics,United States,North America,6.32853
MSFT.US,MSFT,US,Microsoft Corporation,Technology,Software-Infrastructure,United States,North America,5.35457
AMZN.US,AMZN,US,Amazon.com Inc,Consumer Cyclical,Internet Retail,United States,North America,2.64115
GOOGL.US,GOOGL,US,Alphabet Inc Class A,Communication Services,Internet Content & Information,United States,North America,1.71363
BRK-B.US,BRK-B,US,Berkshire Hathaway Inc,Financial Services,Insurance-Diversified,United States,North America,1.63875
...,...,...,...,...,...,...,...,...
FOX.US,FOX,US,Fox Corp Class B,Communication Services,Entertainment,United States,North America,0.01359
DVA.US,DVA,US,DaVita HealthCare Partners Inc,Healthcare,Medical Care Facilities,United States,North America,0.01329
DISH.US,DISH,US,DISH Network Corporation,Communication Services,Telecom Services,United States,North America,0.01017
NWS.US,NWS,US,News Corp B,Communication Services,Entertainment,United States,North America,0.00734


In [227]:
# json_response['General']
# json_response['Technicals']
# json_response['ETF_Data']##['Top 10 Holdings']
# json_response['ETF_Data']['Top_10_Holdings']
# json_response['ETF_Data']['Performance']


holdings = pd.DataFrame(json_response['ETF_Data']['Holdings']).T
performance = pd.DataFrame(list(json_response['ETF_Data']['Performance'].items()))
performance.index = json_response['ETF_Data']['Performance'].keys()
performance = performance.drop(0, axis = 1)

In [92]:
json_response['ETF_Data']['Performance'].keys()

dict_keys(['1y_Volatility', '3y_Volatility', '3y_ExpReturn', '3y_SharpRatio', 'Returns_YTD', 'Returns_1Y', 'Returns_3Y', 'Returns_5Y', 'Returns_10Y'])

In [93]:
performance
holdings.head()

,Code,Exchange,Name,Sector,Industry,Country,Region,Assets_%
AAPL.US,AAPL,US,Apple Inc,Technology,Consumer Electronics,United States,North America,5.14
MSFT.US,MSFT,US,Microsoft Corporation,Technology,Software-Infrastructure,United States,North America,4.68
AMZN.US,AMZN,US,Amazon.com Inc,Consumer Cyclical,Internet Retail,United States,North America,1.91
GOOGL.US,GOOGL,US,Alphabet Inc Class A,Communication Services,Internet Content & Information,United States,North America,1.38
BRK-B.US,BRK-B,US,Berkshire Hathaway Inc,Financial Services,Insurance-Diversified,United States,North America,1.36


In [84]:
tickers = ['SPY', 'QQQ']
d = {}

for i in range(len(tickers)): 

    response = requests.get('https://api.tradier.com/v1/markets/history',
        params={'symbol': tickers[i], 'interval': 'daily', 'start': '2019-01-01', 'end': '2023-01-01'},
        headers={'Authorization': TOKEN, 'Accept': 'application/json'}
    )
    json_response = response.json()
    #print("Response Status Code:", response.status_code)
    i = i + 1
    print("progress", i)
    for symbol in tickers:
        d[symbol] = pd.DataFrame.from_records(json_response['history']['day'], index='date')

        indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
        d[symbol]['macd'] = indicator_macd.macd()
        d[symbol]['macd_diff'] = indicator_macd.macd_diff()
        d[symbol]['macd_signal'] = indicator_macd.macd_signal()

        indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
        d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
        d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
        d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
        d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
        d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

        indicator_vwap = VolumeWeightedAveragePrice(
            high = d[symbol]['high'],
            low = d[symbol]['low'],
            close = d[symbol]['close'],
            volume = d[symbol]['volume'],
            window=14, fillna=True)
        d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

        indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
        d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
        d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
        d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

progress 1
progress 2


In [85]:
df = pd.concat(d.values(), axis=1, keys=d.keys())
dask_df = dd.from_pandas(df, npartitions=6)

In [87]:
df.tail()

SPY                                                         \
              open     high     low   close    volume      macd macd_diff   
date                                                                        
2022-12-23  265.47  268.040  263.53  267.36  39373584 -3.392306 -2.183973   
2022-12-27  266.74  266.850  263.03  263.58  38009005 -4.051257 -2.274339   
2022-12-28  263.17  265.470  259.73  260.10  47139043 -4.798968 -2.417640   
2022-12-29  262.96  267.405  262.25  266.44  45893567 -4.824337 -1.954407   
2022-12-30  263.62  266.410  262.29  266.28  37858277 -4.801999 -1.545655   

                                              ...         QQQ            \
           macd_signal   bb_mavg    bb_hband  ... macd_signal   bb_mavg   
date                                          ...                         
2022-12-23   -1.208334  280.8695  297.487281  ...   -1.208334  280.8695   
2022-12-27   -1.776918  279.9130  298.122859  ...   -1.776918  279.9130   
2022-12-28   -2.381328  278.8895  299.034808  ...   -2.381328  278.8895   
2022-12-29   -2.869930  277.5435  297.233738  ...   -2.869930  277.5435   
2022-12-30   -3.256344  276.1715  294.965473  ...   -3.256344  276.1715   

                                                                          \
              bb_hband    bb_lband bb_hband_ind bb_lband_ind        vwap   
date                                                                       
2022-12-23  297.487281  264.251719          0.0          0.0  278.426559   
2022-12-27  298.122859  261.703141          0.0          0.0  277.342623   
2022-12-28  299.034808  258.744192          0.0          0.0  276.126820   
2022-12-29  297.233738  257.853262          0.0          0.0  275.059930   
2022-12-30  294.965473  257.377527          0.0          0.0  273.997453   

                                            
           stoch_rsi stochrsi_d stochrsi_k  
date                                        
2022-12-23  0.041120   0.078599   0.087737  
2022-12-27  0.000000   0.058491   0.013707  
2022-12-28  0.000000   0.038383   0.013707  
2022-12-29  0.354071   0.048479   0.118024  
2022-12-30  0.348161   0.121936   0.234077  

[5 rows x 34 columns]